In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
sqlContext = SQLContext(sc)

In [3]:
import dask.dataframe as dd
from dask import datasets
import pandas as pd
import datetime
import warnings
import geopandas as gpd
import numpy as np
warnings.filterwarnings('ignore')

In [4]:
!pwd

/home/mingyi/Dropbox/UrbanTemporalNetworks/Data Collection Scripts


In [5]:
zone = gpd.read_file('../Data/NYC Taxi Zones.geojson')
zones = zone['location_id'].unique()


In [6]:
len(zones)

260

In [11]:

def processfromHub(pid, record):
    import csv
    reader = csv.reader(record,delimiter=',')
    for row in reader:
        if len(row) == 18:
            #yellow
            PU = row[7]
            DO = row[8]
            if PU != '' and PU != 'PULocationID' and DO != '' and DO != 'DOLocationID' :
                if row[1] != '':
                    date = row[1].split(' ')[0]
                    year = date.split('-')[0]
                    if year == select_year:
                        if row[3] != '':
                            passenger = int(row[3])
                            #DOLocation
                            if DO in zones:
                                yield ((date,PU,DO),1)

        elif len(row) == 20:
            #green
            PU = row[5]
            DO = row[6]
            if PU != '' and PU != 'PULocationID' and DO != '' and DO != 'DOLocationID' :
                if row[1] != '':
                    date = row[1].split(' ')[0]
                    year = date.split('-')[0]
                    if year == select_year:
                        if row[7] != '':
                            passenger = int(row[7])
                            if DO in zones:
                                yield ((date,PU,DO),1)

        elif len(row) == 6:
            #fhvhv 
            PU = row[3]
            DO = row[4]
            if PU != '' and PU != 'PULocationID' and DO != '' and DO != 'DOLocationID' :
                date = row[1].split(' ')[0]
                if DO != '' and len(DO)<4: 
                    DO = str(int(row[4]))
                    passenger = 1
                    yield ((date,PU,DO),1)

In [13]:
select_year = '2019'
dataDir = '/home/mingyi/Dropbox/UrbanTemporalNetworks/rawData/'+select_year+'/'
processedFileDir = "../processedData/"
processedFile = processedFileDir+select_year+'alltaxizone'
rdd = sc.textFile(dataDir+'*.csv')
counts = rdd.mapPartitionsWithIndex(processfromHub).reduceByKey(lambda x,y:x+y).\
            map(lambda x: (x[0][0],x[0][1],x[0][2],x[1])).cache()
df = sqlContext.createDataFrame(counts,['Date','PULocationID','DOLocationID','vehicle_count'])
df.repartition(1).write.mode("overwrite").\
    format("com.databricks.spark.csv").option("header", "true").save(processedFile)

df = pd.read_csv(processedFile+'/'+[file for file in os.listdir(processedFile) if '.csv' in file and '.crc' not in file][0])



In [15]:
df.to_csv('ridership.csv',index=False)